<a href="https://www.kaggle.com/code/averma111/pytorch-timeseries-pss3e19?scriptVersionId=136495037" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [52]:
%%capture
!pip install pytorch_forecasting

In [53]:
import warnings
import numpy as np
import pandas as pd
import copy
from pathlib import Path
import torch

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

In [54]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/playground-series-s3e19/sample_submission.csv
/kaggle/input/playground-series-s3e19/train.csv
/kaggle/input/playground-series-s3e19/test.csv


In [118]:
train = pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv")
train['date'] = pd.to_datetime(train['date'], errors='coerce')
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['quarter'] = train['date'].dt.quarter
train['weekday'] = train['date'].dt.weekday
train.head()


,id,date,country,store,product,num_sold,year,month,day,quarter,weekday
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63,2017,1,1,1,6
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66,2017,1,1,1,6
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9,2017,1,1,1,6
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59,2017,1,1,1,6
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49,2017,1,1,1,6


In [59]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               136950 non-null  int64  
 1   date             136950 non-null  object 
 2   country          136950 non-null  object 
 3   store            136950 non-null  object 
 4   product          136950 non-null  object 
 5   num_sold         136950 non-null  int64  
 6   sold_by_country  0 non-null       float64
dtypes: float64(1), int64(2), object(4)
memory usage: 7.3+ MB


In [87]:
product_by_sold=train.groupby(['product'])['num_sold'].sum().reset_index()
product_by_sold

,product,num_sold
0,Using LLMs to Improve Your Coding,6069281
1,Using LLMs to Train More LLMs,5993975
2,Using LLMs to Win Friends and Influence People,932564
3,Using LLMs to Win More Kaggle Competitions,5068313
4,Using LLMs to Write Better,4604192


In [101]:
store_by_sold=train.groupby(['store'])['num_sold'].sum().reset_index()
store_by_sold

,store,num_sold
0,Kagglazon,15661335
1,Kaggle Learn,2755934
2,Kaggle Store,4251056


In [102]:
country_by_sold=train.groupby(['country'])['num_sold'].sum().reset_index()
country_by_sold

,country,num_sold
0,Argentina,1683244
1,Canada,7013114
2,Estonia,3577319
3,Japan,6013786
4,Spain,4380862


In [103]:
date_by_sold=train.groupby(['date'])['num_sold'].sum().reset_index()
date_by_sold

,date,num_sold
0,2017-01-01,20086
1,2017-01-02,15563
2,2017-01-03,15039
3,2017-01-04,14516
4,2017-01-05,14083
...,...,...
1821,2021-12-27,16724
1822,2021-12-28,18507
1823,2021-12-29,20110
1824,2021-12-30,20156


In [113]:
year_by_sold=train.groupby(['year'])['num_sold'].sum().reset_index()
year_by_sold

,year,num_sold
0,2017,4451213
1,2018,4723963
2,2019,4522172
3,2020,4086498
4,2021,4884479


In [114]:
month_by_sold=train.groupby(['month'])['num_sold'].sum().reset_index()
month_by_sold

,month,num_sold
0,1,2042783
1,2,1782267
2,3,1905982
3,4,1764963
4,5,1866246
5,6,1779787
6,7,1858402
7,8,1867996
8,9,1833097
9,10,1924133


In [97]:
fig = px.bar(product_by_sold,x='product',y='num_sold')
fig.show()

In [92]:
fig = px.bar(store_by_sold,x='store',y='num_sold')
fig.show()

In [94]:
fig = px.bar(country_by_sold,x='country',y='num_sold')
fig.show()

In [100]:
fig = px.line(date_by_sold,x='date',y='num_sold')
fig.show()

In [109]:
fig = px.bar(year_by_sold,x='year',y='num_sold')
fig.show()

In [112]:
fig = px.bar(month_by_sold,x='month',y='num_sold')
fig.show()

In [119]:
train.head()

,id,date,country,store,product,num_sold,year,month,day,quarter,weekday
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63,2017,1,1,1,6
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66,2017,1,1,1,6
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9,2017,1,1,1,6
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59,2017,1,1,1,6
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49,2017,1,1,1,6


In [123]:
train.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold', 'year',
       'month', 'day', 'quarter', 'weekday'],
      dtype='object')

In [133]:
max_prediction_length = 1
max_encoder_length = 12
training_cutoff = train['month'].max() - max_prediction_length

training = TimeSeriesDataSet(
    train[lambda x: x['month'] <= training_cutoff],
    time_idx='id',
    target='num_sold',
    group_ids=['store','product','country'],
    min_encoder_length=0,  
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=['store','country','product'],
    #static_reals=['store','country','product'],
    time_varying_known_categoricals=[],  
    time_varying_known_reals=['month','day','quarter','year','weekday'],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[],
    categorical_encoders={'store': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
                          'country':pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
                          'product':pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True)},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
    
)

In [134]:
validation = TimeSeriesDataSet.from_dataset(training,train, predict=True, stop_randomization=True)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/pytorch_forecasting/data/encoders.py:331 in transform    │
│                                                                                                  │
│    328 │   │   │   │   encoded = [self.classes_.get(v, na_fill_value) for v in y]                │
│    329 │   │   │   else:                                                                         │
│    330 │   │   │   │   try:                                                                      │
│ ❱  331 │   │   │   │   │   encoded = [self.classes_[v] for v in y]                               │
│    332 │   │   │   │   except KeyError as e:                                                     │
│    333 │   │   │   │   │   raise KeyError(                                                       │
│    334 │   │   │   │   │   │   f"Unknown category '{e.args[0]}' encountered. Set `add_nan=True`  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pytorch_forecasting/data/encoders.py:331 in <listcomp>   │
│                                                                                                  │
│    328 │   │   │   │   encoded = [self.classes_.get(v, na_fill_value) for v in y]                │
│    329 │   │   │   else:                                                                         │
│    330 │   │   │   │   try:                                                                      │
│ ❱  331 │   │   │   │   │   encoded = [self.classes_[v] for v in y]                               │
│    332 │   │   │   │   except KeyError as e:                                                     │
│    333 │   │   │   │   │   raise KeyError(                                                       │
│    334 │   │   │   │   │   │   f"Unknown category '{e.args[0]}' encountered. Set `add_nan=True`  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 976

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 validation = TimeSeriesDataSet.from_dataset(training,train, predict=True, stop_randomiza     │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pytorch_forecasting/data/timeseries.py:1154 in           │
│ from_dataset                                                                

In [ ]:
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [ ]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=1,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=1,  # 7 quantiles by default
    loss=pytorch_forecasting.metrics.RMSE(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# find optimal learning rate
res = trainer.tuner.lr_find(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=0.1,
    min_lr=1e-7,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-7, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  
logger = TensorBoardLogger("lightning_logs") 

trainer = pl.Trainer(
    max_epochs=30,
    gpus=1,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=5e-7,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=1, 
    loss=pytorch_forecasting.metrics.RMSE(),
    log_interval=10,  
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# calcualte root mean squared error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
val_predictions = best_tft.predict(val_dataloader)

In [ ]:
criterion = nn.MSELoss()
torch.sqrt(criterion(actuals,val_predictions))

In [ ]:
df_test = pd.read_csv('/kaggle/input/playground-series-s3e19/test.csv')
df_test=df_test.drop(['ID'],axis=1)
df_test['date_block_num']=34
df_test['item_cnt_month']=0
# decoder_data=df_test
# encoder_data = df_train[lambda x: x.date_block_num > x.date_block_num.max() - max_encoder_length]
# new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
pred,x=best_tft.predict(df_test,return_x=True)